In [1]:
import numpy as np

from keras.models import Sequential, Model
from keras.layers import Input, Dense, TimeDistributed, merge, Lambda
from keras.layers.core import *
from keras.layers.convolutional import *
from keras.layers.recurrent import *

import tensorflow as tf
from keras import backend as K

from keras.activations import relu
from functools import partial
clipped_relu = partial(relu, max_value=5)

Using TensorFlow backend.


In [2]:
def max_filter(x):
    # Max over the best filter score (like ICRA paper)
    max_values = K.max(x, 2, keepdims=True)
    max_flag = tf.greater_equal(x, max_values)
    out = x * tf.cast(max_flag, tf.float32)
    return out

def channel_normalization(x):
    # Normalize by the highest activation
    max_values = K.max(K.abs(x), 2, keepdims=True)+1e-5
    out = x / max_values
    return out

def WaveNet_activation(x):
    tanh_out = Activation('tanh')(x)
    sigm_out = Activation('sigmoid')(x)  
    return Merge(mode='mul')([tanh_out, sigm_out])


In [3]:
def ED_TCN(n_nodes, conv_len, n_classes, n_feat, max_len, 
            loss='categorical_crossentropy', causal=False, 
            optimizer="rmsprop", activation='norm_relu',
            return_param_str=False):
    n_layers = len(n_nodes)

    inputs = Input(shape=(max_len,n_feat))
    model = inputs

    # ---- Encoder ----
    for i in range(n_layers):
        # Pad beginning of sequence to prevent usage of future data
        if causal: model = ZeroPadding1D((conv_len//2,0))(model)
        model = Convolution1D(n_nodes[i], conv_len, border_mode='same')(model)
        if causal: model = Cropping1D((0,conv_len//2))(model)

        model = SpatialDropout1D(0.3)(model)
        
        if activation=='norm_relu': 
            model = Activation('relu')(model)            
            model = Lambda(channel_normalization, name="encoder_norm_{}".format(i))(model)
        elif activation=='wavenet': 
            model = WaveNet_activation(model) 
        else:
            model = Activation(activation)(model)            
        
        model = MaxPooling1D(2)(model)

    # ---- Decoder ----
    for i in range(n_layers):
        model = UpSampling1D(2)(model)
        if causal: model = ZeroPadding1D((conv_len//2,0))(model)
        model = Convolution1D(n_nodes[-i-1], conv_len, border_mode='same')(model)
        if causal: model = Cropping1D((0,conv_len//2))(model)

        model = SpatialDropout1D(0.3)(model)

        if activation=='norm_relu': 
            model = Activation('relu')(model)
            model = Lambda(channel_normalization, name="decoder_norm_{}".format(i))(model)
        elif activation=='wavenet': 
            model = WaveNet_activation(model) 
        else:
            model = Activation(activation)(model)

    # Output FC layer
    model = TimeDistributed(Dense(n_classes, activation="softmax" ))(model)
    
    model = Model(input=inputs, output=model)
    model.compile(loss=loss, optimizer=optimizer, sample_weight_mode="temporal", metrics=['accuracy'])

    if return_param_str:
        param_str = "ED-TCN_C{}_L{}".format(conv_len, n_layers)
        if causal:
            param_str += "_causal"
    
        return model, param_str
    else:
        return model

In [5]:
save_predictions = [False, True][1]
viz_predictions = [False, True][1]
viz_weights = [False, True][0]

# dataset = ["50Salads", "JIGSAWS", "MERL", "GTEA"][0]

# How many latent states/nodes per layer of network
# Only applicable to the TCNs. The ECCV and LSTM  model suses the first element from this list.
n_nodes = [64, 96]
nb_epoch = 200
video_rate = 3
# conv = {'50Salads':25, "JIGSAWS":20, "MERL":5, "GTEA":25}[dataset]


n_classes = data.n_classes
train_lengths = [x.shape[0] for x in X_train]
test_lengths = [x.shape[0] for x in X_test]
n_train = len(X_train)
n_test = len(X_test)

n_feat = data.n_features
print("# Feat:", n_feat)

model_type = "ED-TCN"

NameError: name 'data' is not defined

In [ ]:
# --------- CVPR model ----------
if model_type in ["tCNN", "ED-TCN", "DilatedTCN", "TDNN", "LSTM"]:
    # Go from y_t = {1...C} to one-hot vector (e.g. y_t = [0, 0, 1, 0])
    Y_train = [np_utils.to_categorical(y, n_classes) for y in y_train]
    Y_test = [np_utils.to_categorical(y, n_classes) for y in y_test]

    # In order process batches simultaneously all data needs to be of the same length
    # So make all same length and mask out the ends of each.
    n_layers = len(n_nodes)
    max_len = max(np.max(train_lengths), np.max(test_lengths))
    max_len = int(np.ceil(max_len / (2**n_layers)))*2**n_layers
    print("Max length:", max_len)

    X_train_m, Y_train_, M_train = utils.mask_data(X_train, Y_train, max_len, mask_value=-1)
    X_test_m, Y_test_, M_test = utils.mask_data(X_test, Y_test, max_len, mask_value=-1)
        
    if model_type == "ED-TCN":
        model, param_str = tf_models.ED_TCN(n_nodes, conv, n_classes, n_feat, max_len, causal=causal, 
                                        activation='norm_relu', return_param_str=True) 
    
    AP_train = model.predict(X_train_m, verbose=0)
    AP_test = model.predict(X_test_m, verbose=0)
    AP_train = utils.unmask(AP_train, M_train)
    AP_test = utils.unmask(AP_test, M_test)

    P_train = [p.argmax(1) for p in AP_train]
    P_test = [p.argmax(1) for p in AP_test]

In [4]:
n_nodes = [64, 96]
conv = 25
n_classes = 12
n_feat = 128
max_len = 552

model, param_str = ED_TCN(n_nodes, conv, n_classes, n_feat, max_len, causal=False, activation='norm_relu', return_param_str=True)
net = model.summary()
print(net)

/Users/daichi/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 25, padding="same")`
  
/Users/daichi/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(96, 25, padding="same")`
  


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 552, 128)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 552, 64)           204864    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 552, 64)           0         
_________________________________________________________________
activation_1 (Activation)    (None, 552, 64)           0         
_________________________________________________________________
encoder_norm_0 (Lambda)      (None, 552, 64)           0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 276, 64)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 276, 96)           153696    
__________

/Users/daichi/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/ipykernel_launcher.py:33: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(96, 25, padding="same")`
/Users/daichi/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/ipykernel_launcher.py:33: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 25, padding="same")`
/Users/daichi/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/ipykernel_launcher.py:49: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ti...)`


In [6]:
from keras.utils.vis_utils import plot_model

plot_model(model, "model125.png", show_layer_names=False)

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.